<a href="https://colab.research.google.com/github/EoghanDoyle/Thesis/blob/main/Artefact_Netflix_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619422 sha256=ce3fb0126d1fc94cf2192072088323c9614a74f6cae0ab37706ef655e7c210fd
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
#Imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection.split import train_test_split
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate
from surprise import NormalPredictor



In [ ]:
#-------------------------------------------------------Data Preprocessing -------------------------------------------------------


# Importing dataset and examining it
movies_dataset = pd.read_csv("/content/drive/My Drive/DBS/Netflix_dataset.csv")
# print(movies_dataset.head())
# print(movies_dataset.shape)
# print(movies_dataset.info())
# print(movies_dataset.describe())

#gets total numbers of ratings for each movie
total_ratings = pd.DataFrame(movies_dataset.groupby('MovieId')['Rating'].count())
print(total_ratings.head(10))

total_ratings.to_csv('/content/drive/My Drive/DBS/Netflix_total_ratings.csv')

#merge ratings df and original df
combined_dataset = movies_dataset.merge(total_ratings,on='MovieId', how='left')
print(combined_dataset.head())

min_movie_ratings = 2 #a movie has was rated at least 
min_user_ratings =  5 #a user rated movies at least

movies_filtered = combined_dataset.groupby("MovieId").filter(lambda x: x['MovieId'].count() >= min_movie_ratings)
movies_filtered = movies_filtered.groupby("UserId").filter(lambda x: x['UserId'].count() >= min_user_ratings)

final_movies_dataset = movies_filtered.drop(['Rating_y', 'Unnamed: 0'],axis=1)

final_movies_dataset.to_csv('/content/drive/My Drive/DBS/Processed_Real_netflix_dataset.csv')

reader = Reader()
#dataset creation
data = Dataset.load_from_df(final_movies_dataset, reader)

         Rating
MovieId        
1           243
2            66
3            48
4            28
5            24
6           101
7            38
8             4
9            12
10           98
   Unnamed: 0  UserId  MovieId  Rating_x  Rating_y
0      492068    3026     2134         3        58
1       33921     229     1735         5        36
2      225139    1361     1207         5        87
3      459670    2832     1035         3        86
4      645914    3887      337         3        76


In [ ]:
#------------------------------------------------------------------ KNN --------------------------------------------------------
param_grid = {'k':[10,20,30,40,50,60] , 
              'sim_options': {'name': ['cosine', 'pearson'] }
              }

gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best KNN RMSE score attained: ", gs.best_score['rmse'])
print("Best KNN MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computi

In [ ]:
#------------------------------------------------------------------ SVD ---------------------------------------------------------
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best SVD RMSE score attained: ", gs.best_score['rmse'])
print("Best SVD MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])

Best SVD RMSE score attained:  0.946492950899696
Best SVD MAE score attained:  0.7507238891107424
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}


In [ ]:
#------------------------------------------------------------------ SVD++ ---------------------------------------------------------
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print("Best SVD++ RMSE score attained: ", gs.best_score['rmse'])
print("Best SVD++ MAE score attained: ", gs.best_score['mae'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
print(gs.best_params['mae'])


Best SVD++ RMSE score attained:  0.9465252170896434
Best SVD++ MAE score attained:  0.7507221071374255
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}


In [ ]:
#------------------------------------------------------------------ Random ---------------------------------------------------------
Random = NormalPredictor()
results = cross_validate(Random, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
print(results)


{'test_rmse': array([1.50294294, 1.50002149, 1.50011313, 1.50509387, 1.50771282]), 'test_mae': array([1.20235141, 1.20362358, 1.20406339, 1.20579343, 1.20709133]), 'fit_time': (0.13940644264221191, 0.1843719482421875, 0.18368196487426758, 0.18352460861206055, 0.1842184066772461), 'test_time': (0.14230942726135254, 0.23339223861694336, 0.14002490043640137, 0.2276778221130371, 0.14281558990478516)}
